In [1]:
from dam_break import setup as clawpack_setup
import numpy as np
import matplotlib.pyplot as plt

def generate_claw():
    claw = clawpack_setup()
    claw.verbosity= 0
    claw.run()
    return claw

def construct_data_dictionaries(claw):
    frames = claw.frames
    data_dict = {}
    data_dict["x"] = (claw.frames[0].state.grid.x.centers).reshape(-1,1)
    t_list = []
    usol_list = []
    for i in range(len(frames)):
        t_list.append(frames[i].t)
        usol_list.append(frames[i].state.q.T)

    data_dict["t"] = np.array(t_list).reshape(-1,1)
    data_dict["usol"] = np.array(usol_list)
    return data_dict

def write_data_dict():
    #Meant to be run locally
    claw = generate_claw()
    data_dict = construct_data_dictionaries(claw)
    np.save("../SWEs_data_CLAWPACK.npy", data_dict)

def read_data_dict():
    #Meant to be run locally
    data_dict = np.load("../SWEs_data_CLAWPACK.npy", allow_pickle=True).item()
    return data_dict

In [2]:
write_data_dict()

In [7]:
data = read_data_dict()

In [8]:
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol'])

In [9]:
t.shape

(101, 1)

In [12]:
Exact.shape

(101, 1000, 2)

In [14]:
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))

In [15]:
X_star.shape

(101000, 2)

In [25]:
Exact.shape

(101, 1000, 2)

In [30]:
q = Exact[:,:,1].flatten()[:,None]
h = Exact[:,:,0].flatten()[:,None]

In [33]:
u_star = np.hstack((h,q))

In [34]:
u_star.shape

(101000, 2)

In [35]:
q.shape

(101000, 1)

In [37]:
Exact[0:1,:,:].shape

(1, 1000, 2)

In [68]:
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T)) # each row contains [x,t=0]
uu1 = Exact[0:1,:,:] # each row contains u(x,t=0)
xx2 = np.hstack((X[:,0:1], T[:,0:1])) # each row contains [x=-1,t]
uu2 = Exact[:,0:1,:] # each row contains u(x=-1,t)
xx3 = np.hstack((X[:,-1:], T[:,-1:])) # each row contains [x=1,t]
uu3 = Exact[:,-1:,:] # each row contains u(x=1,t)

In [72]:
#Taking care of shapes
hh1 = uu1[:,:,0].flatten()[:,None]
qq1 = uu1[:,:,1].flatten()[:,None]
# Expand dimensions to shape (1000,1,1)
hh1_expanded = hh1[:, :, np.newaxis]
qq1_expanded = qq1[:, :, np.newaxis]
uu1 = np.concatenate((hh1_expanded, qq1_expanded), axis=2)




(1000, 1, 2)

In [51]:
#uu1.shape
uu2.shape
uu3.shape

(101, 1, 2)

In [45]:
#Stack vertically to obtain training data
X_u_train = np.vstack([xx1, xx2, xx3])
X_u_train.shape


(1202, 2)

In [73]:
u_train = np.vstack([uu1, uu2, uu3])

In [78]:
u_train=np.squeeze(u_train, axis=1).shape
u_train.shape

AxisError: axis 1 is out of bounds for array of dimension 1

In [75]:
X_u_train.shape

(1202, 2)

In [80]:
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T)) # each row contains [x,t=0]
uu1 = Exact[0:1,:,:] # each row contains u(x,t=0)
xx2 = np.hstack((X[:,0:1], T[:,0:1])) # each row contains [x=-1,t]
uu2 = Exact[:,0:1,:] # each row contains u(x=-1,t)
xx3 = np.hstack((X[:,-1:], T[:,-1:])) # each row contains [x=1,t]
uu3 = Exact[:,-1:,:] # each row contains u(x=1,t)

#Taking care of shapes
hh1 = uu1[:,:,0].flatten()[:,None]
qq1 = uu1[:,:,1].flatten()[:,None]
# Expand dimensions to shape (1000,1,1)
hh1_expanded = hh1[:, :, np.newaxis]
qq1_expanded = qq1[:, :, np.newaxis]
uu1 = np.concatenate((hh1_expanded, qq1_expanded), axis=2)


X_u_train = np.vstack([xx1, xx2, xx3])
u_train = np.vstack([uu1, uu2, uu3])
u_train=np.squeeze(u_train, axis=1)

In [81]:
X_u_train.shape

(1202, 2)

In [82]:
u_train.shape

(1202, 2)